# Lens Tutorial


*Lens* is a library for exploring data in Pandas DataFrames. It computes single
column summary statistics and estimates the correlation between columns.

We wrote *Lens* when we realised that the initial steps of acquiring a new
dataset were almost formulaic: what data type is in this column? How many null
values are there? Which columns are correlated? What's the distribution of this
value? Lens calculates all this for you, and provides convenient visualisation
of this information.

You can use *Lens* to analyse new datasets as well as using it to compare how
DataFrames change over time.

## Using lens

To start using *Lens* you need to import the library:

In [1]:
import lens

*Lens* has two key functions; ``lens.summarise`` for generating a Lens Summary from a DataFrame and
``lens.explore`` for visualising the results of a summary.

For this tutorial we are going to use *Lens* to analyse the Room Occupancy
dataset provided in the [Machine Learning Repository of UC Irvine](https://archive.ics.uci.edu/ml/datasets/Occupancy+Detection+). It includes 
ambient information about a room such as Temperature, Humidity,
Light, CO2 and whether it was occupied. The goal is to
predict occupancy based on the room measurements.

To read it into pandas use:

In [2]:
import pandas as pd
df = pd.read_csv('http://asi-datasets.s3.amazonaws.com/room_occupancy/room_occupancy.csv')

# Split a numerical variable to have additional categorical variables
df['Humidity_cat'] = pd.cut(df['Humidity'], 5,
                            labels=['low', 'medium-low', 'medium',
                                    'medium-high', 'high']).astype('str')

In [3]:
print('Number of rows in dataset: {}'.format(len(df.index)))
df.head()

Number of rows in dataset: 8143


,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy,Humidity_cat
1,2015-02-04 17:51:00,23.18,27.2720,426.0,721.25,0.004793,1,medium
2,2015-02-04 17:51:59,23.15,27.2675,429.5,714.00,0.004783,1,medium
3,2015-02-04 17:53:00,23.15,27.2450,426.0,713.50,0.004779,1,medium
4,2015-02-04 17:54:00,23.15,27.2000,426.0,708.25,0.004772,1,medium
5,2015-02-04 17:55:00,23.10,27.2000,426.0,704.50,0.004757,1,medium


## Creating the summary

When you have a DataFrame that you'd like to analyse the first thing to do is
to create a Lens ``Summary`` object.

In [4]:
ls = lens.summarise(df)

The `summarise` function takes a DataFrame and returns a Lens ``Summary`` object. The
time this takes to run is dependent on both the number of rows and the number of
columns in the DataFrame. It will use all cores available on the machine, so you 
might want to use a SherlockML instance with more cores to speed up the computation 
of the summary. There are additional optional parameters that can be
passed in. Details of these can be found in the [summarise API docs](http://docs.sherlockml.com/lens/summarise_api.html#lens.summarise.summarise).

Given that creating the summary is computationally intensive, *Lens* provides a way to save this summary to a JSON file on disk and recover a saved summary through the `to_json` and `from_json` methods of `lens.summary`. This allows to store it for future analysis or to share it with collaborators:

In [5]:
# Saving to JSON
ls.to_json('room_occupancy_lens_summary.json')

# Reading from a file
ls_from_json = lens.Summary.from_json('room_occupancy_lens_summary.json')

The `LensSummary` object contains the information computed from the dataset and provides methods to access both column-wise and whole dataset information. It is designed to be used programatically, and information about the methods can be accessed in the [LensSummary API docs](http://docs.sherlockml.com/lens/summarise_api.html#lens.summarise.Summary).

In [6]:
print(ls.columns)

['date', 'Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio', 'Occupancy', 'Humidity_cat']


## Create explorer

Lens provides a function that converts a Lens Summary into an `Explorer` object.
This can be used to see the summary information in tabular form and to display
plots.

In [7]:
explorer = lens.explore(ls)

Coming back to our room occupancy dataset, the first thing that we'd like to
know is a high-level overview of the data.

### Describe

To show a general description of the DataFrame call the `describe` function.
This is similar to Pandas' ``DataFrame.describe`` but also shows information for non-numeric columns.

In [8]:
explorer.describe()

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy,Humidity_cat
desc,None,numeric,numeric,numeric,numeric,numeric,categorical,categorical
dtype,object,float64,float64,float64,float64,float64,int64,object
notnulls,8143,8143,8143,8143,8143,8143,8143,8143
nulls,0,0,0,0,0,0,0,0
unique,8143,265,1325,889,2282,3583,2,5


We can see that our dataset has 8143 rows and all the rows are complete. This
is a very clean dataset! It also tells us the columns and their types, including a `desc` field that explains how *Lens* will treat this column.

### Column details

To see type-specific column details, use the `column_details` method. Used on a numeric column such as `Temperature`, it provides summary statistics for the data in that column, including minimun, maximum, mean, median, and standard deviation.

In [9]:
explorer.column_details('Temperature')

,Temperature
desc,numeric
dtype,float64
min,19.0
max,23.18
mean,20.6190836403
median,20.39
std,1.01691644111
sum,167901.198083
IQR,1.69


We saw in the ouput of `explorer.describe()` that `Occupancy`, our target variable, is a categorical column with two unique values. With `explorer.column_details` we can obtain a frequency table for these two categories - empty (0) or occupied (1):

In [10]:
explorer.column_details('Occupancy')

item,frequency
0,6414
1,1729


### Correlation

As a first step in exploring the relationships between the columns we can look at the correlation coefficients. `explorer.correlation()` returns a Spearman rank-order correlation coefficient matrix in tabular form.

In [11]:
explorer.correlation()

,Humidity,HumidityRatio,Temperature,CO2,Light,Occupancy
Humidity,1.0,0.9403266291569079,-0.19338923334162547,0.22351889930108668,0.0070653086819316094,0.1292350876247768
HumidityRatio,0.9403266291569079,1.0,0.10476509821418747,0.3783569229168182,0.16905712790190097,0.25583595029547784
Temperature,-0.19338923334162547,0.10476509821418747,1.0,0.6369060817084772,0.5652372559460639,0.5328303325204367
CO2,0.22351889930108668,0.3783569229168182,0.6369060817084772,1.0,0.4731030126223473,0.6566512850978677
Light,0.0070653086819316094,0.16905712790190097,0.5652372559460639,0.4731030126223473,1.0,0.8046454034169337
Occupancy,0.1292350876247768,0.25583595029547784,0.5328303325204367,0.6566512850978677,0.8046454034169337,1.0


However, parsing a correlation table becomes difficult when there are many columns in the dataset. To get a better overview, we can plot the correlation matrix as a heatmap, which immediately highlights a group of columns correlated with `Occupancy`: `Temperature`, `Light`, and `CO2`.

In [13]:
explorer.correlation_plot()

### Distribution and Cumulative Distribution

We can explore the distribution of numerical variables through the `distribution_plot` and `cdf_plot` functions:

In [14]:
explorer.distribution_plot('Temperature')

In [15]:
explorer.cdf_plot('Temperature')

### Pairwise plot

Once we know that certain columns might be correlated, it is useful to visually explore that correlation. This would typically be done through a scatter plot, and *Lens* has computed a 2D Kernel Density Estimate of the scatter plot that can be accessed through `pairwise_density_plot`.

In [16]:
explorer.pairwise_density_plot('Temperature', 'Humidity')

`pairwise_density_plot` can also show the relationship between a numeric column and a categorical column. In this case, a 1D KDE is computed for each of the categories in the categorical column.

In [17]:
explorer.pairwise_density_plot('Temperature', 'Occupancy')

### Crosstab

The pairwise relationship between two categorical variables can also be seen as a cross-tabulation: how many observations exist in the dataset of the combination of categories in the two variables. This can be seen as a table or as a plot, which can be useful when the number of categories is very large.

In [18]:
explorer.crosstab('Occupancy', 'Humidity_cat')

,0,1
high,143,243
low,2127,425
medium,1880,428
medium-high,1420,310
medium-low,844,323


In [19]:
explorer.pairwise_density_plot('Occupancy', 'Humidity_cat')

## Interactive widget

An alternative way of quickly exploring the plots available in *Lens* is through a Jupyter widget provided by `lens.interactive_explore`. Creating it is as easy as running this function on a *Lens* `Summary`.

Note that if you are reading this tutorial through the online docs the output of the following cell will not be interactive as it needs to run within a notebook. Download the notebook from the links below to try out the interactive explorer!

In [19]:
lens.interactive_explore(ls)

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
